In [102]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [103]:
env = GridworldEnv()

In [143]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.0001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_ = np.zeros(env.nS)
    delta = 0
    k = 0
    while True:
        delta = 0
        k+=1
        for s in range(env.nS):
            total = 0
            v = V[s]
            for a in range(env.nA):
                for prob, next_state, reward, _ in env.P[s][a]:
                    total += policy[s][a] * prob * (reward + discount_factor * V_[next_state])
            V[s] = total
            delta = max(delta , abs(V[s] - v))
            
        V_ = V
        if delta < theta:
            break
    return np.asarray(V)

In [144]:
env.P[0][0][0]

(1.0, 0, 0.0, True)

In [145]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [146]:
v

array([  0.        , -13.99934883, -19.99906386, -21.9989696 ,
       -13.99934883, -17.99920093, -19.99913239, -19.99914232,
       -19.99906386, -19.99913239, -17.9992679 , -13.9994534 ,
       -21.9989696 , -19.99914232, -13.9994534 ,   0.        ])

In [147]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)